In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [3]:
df = pd.read_parquet("/beegfs/ws/0/s4610340-energy_behavior/yahor/kaggle-predict_energy_behavior_of_prosumers/data/processed/train/make_features/df_features.parquet", engine="fastparquet")
df = df.loc[df["is_consumption"]==0]
df["target_per_capacity"] = df["target"] / df["installed_capacity"]
df

,county,is_business,product_type,is_consumption,datetime,day,weekday,month,year,segment,...,target_per_eic_ratio_48_216,target_per_eic_ratio_72_240,target_per_eic_ratio_48_72,target_per_eic_all_type_sum_ratio_48_72,target_per_eic_all_type_sum_ratio_168_336,target_per_eic_all_county_type_sum_ratio_48_72,target_per_eic_all_county_type_sum_ratio_168_336,is_country_holiday,target,target_per_capacity
row_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,0,2021-09-01 00:00:00,1,3,9,2021,0_0_1_0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.713,NaN
2,0,0,2,0,2021-09-01 00:00:00,1,3,9,2021,0_0_2_0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.000,NaN
4,0,0,3,0,2021-09-01 00:00:00,1,3,9,2021,0_0_3_0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2.904,NaN
6,0,1,0,0,2021-09-01 00:00:00,1,3,9,2021,0_1_0_0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.000,NaN
8,0,1,1,0,2021-09-01 00:00:00,1,3,9,2021,0_1_1_0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018342,15,0,1,0,2023-05-31 23:00:00,31,3,5,2023,15_0_1_0,...,0.083333,0.134615,0.086207,0.24413,1.848061,0.924768,0.827064,False,0.004,0.000010
2018344,15,0,3,0,2023-05-31 23:00:00,31,3,5,2023,15_0_3_0,...,0.912284,4.372575,0.243014,0.24413,1.848061,0.924768,0.827064,False,2.287,0.001123
2018346,15,1,0,0,2023-05-31 23:00:00,31,3,5,2023,15_1_0_0,...,0.000000,0.000000,0.000000,0.00000,0.000000,1.170799,0.647868,False,0.000,0.000000


In [17]:
def explore_feature_vs_target(x: str, win=10, step=10, val_thr=0.0):
    series = df.set_index(df[x] / df[x].max()).sort_index()["target_per_capacity"]
    series = series.loc[series.index>val_thr]
    series_rolling_median = series.rolling(win).median()[::step]
    series_rolling_q75 = series.rolling(win).quantile(0.75)[::step]
    series_rolling_q25 = series.rolling(win).quantile(0.25)[::step]

    fig = go.Figure(
        [
            go.Scattergl(
                x=series_rolling_median.index, 
                y=series_rolling_median, 
                mode="markers", 
                hoverinfo="skip",
            ),
            go.Scatter(
                x=series_rolling_q75.index,
                y=series_rolling_q75,
                mode='lines',
                marker=dict(color="#444"),
                line=dict(width=0),
                showlegend=False
            ),
            go.Scatter(
                x=series_rolling_q25.index,
                y=series_rolling_q25,
                marker=dict(color="#444"),
                line=dict(width=0),
                mode='lines',
                fillcolor='rgba(68, 68, 68, 0.3)',
                fill='tonexty',
                showlegend=False
            )
        ]
    )

    return fig


explore_feature_vs_target("surface_solar_radiation_downwards_forecast", 250, 100)

In [20]:
explore_feature_vs_target("snowfall_historical", 100, 50, 0.0)

In [9]:
df["rain_forecast"].clip(0).describe()

count    1.007457e+06
mean     4.465757e-05
std      1.662766e-04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      9.827024e-06
max      3.942772e-03
Name: rain_forecast, dtype: float64

In [23]:
(df["total_precipitation_forecast"] - df["snowfall_forecast"]*334/(100*1000)).describe()

count    1.007457e+06
mean     8.307369e-05
std      2.075521e-04
min     -5.795658e-06
25%      0.000000e+00
50%      7.737336e-06
75%      6.017493e-05
max      3.942772e-03
dtype: float64